# ETL Pterodactyl Minecraft Application

### Index

- Install requierements
- Import libraries and setup key variables
- Setup directories, functions and folder creation
- Get Pterodactyl Application information
- Upload csv table files into Postgres

## Install requierements

In [ ]:
pip install -r requirements.txt

## Import libraries and setup key variables
Remember to add you own credentials in the .env file for them to be loaded here

In [ ]:
import datetime, csv, os
from sqlalchemy import create_engine, text
from pydactyl import PterodactylClient
from dotenv import load_dotenv
from shlink import Shlink
import pandas as pd

# Load .env file credentials
load_dotenv()

# Database connection
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
database = os.getenv('POSTGRES_DATABASE')
username = os.getenv('POSTGRES_USERNAME')
password = os.getenv('POSTGRES_PASSWORD')
connection = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Shlink connection
shlink_url = os.getenv('SHLINK_URL')
shlink_api_key = os.getenv('SHLINK_API_KEY')

# Connecto to Shlink API
shlink = Shlink(url=shlink_url, api_key=shlink_api_key)

## Setup directories, functions and folder creation

In [ ]:
# Create folders, variables or whatever you need here before coding below

## Get Pterodactyl Application information
About: locations, nodes, nests, eggs, servers, clients

In [ ]:
# Extracting data from Shlink API

## Upload csv table files into Postgres

In [ ]:
engine = create_engine(connection)

for file_table in os.listdir(server_app_folder):

    # Setup database variables
    ID = 'id'
    TABLE = file_table.split('.')[0]
    TABLE_UPDATE = TABLE + '_update'
    IS_ACTIVE_TABLE = 'is_active_table'
    SCHEMA = 'pterodactyl'
    SCHEMA_UPDATE = 'pterodactyl_update'
    
    # Reading of the file_table
    df = pd.read_csv(os.path.join(server_app_folder, file_table))

    # Start connection with database
    with engine.connect() as conn:
        # Start a new transaction
        trans = conn.begin()

        try:
            # Load ID from database
            result = conn.execute(text(f'SELECT "{ID}" FROM {SCHEMA}.{TABLE}'))
            db = pd.DataFrame(result.fetchall(), columns=result.keys())

            # Compare ID
            sameID = db[ID].isin(df[ID])

            toUpdate = df[df[ID].isin(db[ID][sameID])]
            toIngest = df[~df[ID].isin(db[ID][sameID])]
            toDelete = db[~db[ID].isin(df[ID])]

            # Insert the DataFrame into a table
            toIngest.to_sql(TABLE, conn, schema=SCHEMA, if_exists='append', index=False)

            # Insert the updatable DataFrame into the TABLE_UPDATE table
            toUpdate.to_sql(TABLE_UPDATE, conn, schema=SCHEMA_UPDATE, if_exists='append', index=False)

            # Define and execute the following queries
            conn.execute(text(f'DELETE FROM {SCHEMA}.{TABLE} WHERE "{ID}" IN (SELECT "{ID}" FROM {SCHEMA_UPDATE}.{TABLE_UPDATE});'))
            conn.execute(text(f'INSERT INTO {SCHEMA}.{TABLE} SELECT * FROM {SCHEMA_UPDATE}.{TABLE_UPDATE};'))
            conn.execute(text(f'TRUNCATE TABLE {SCHEMA_UPDATE}.{TABLE_UPDATE};'))

            # Update column "is_active" from tables when data is deleted from Pterodactyl App
            toDelete.to_sql(IS_ACTIVE_TABLE, conn, schema=SCHEMA_UPDATE, if_exists='append', index=False)
            conn.execute(text(f'UPDATE {SCHEMA}.{TABLE} SET is_active = false WHERE "{ID}" IN (SELECT * FROM {SCHEMA_UPDATE}.{IS_ACTIVE_TABLE});'))
            conn.execute(text(f'TRUNCATE TABLE {SCHEMA_UPDATE}.{IS_ACTIVE_TABLE};'))

            # Commit the transaction
            trans.commit()

        except Exception as e:
            # Rollback the transaction on exception
            print('!!! [ERROR IN DATABASE QUERIES] !!!')
            trans.rollback()
            print('Transaction has been rolled back')
            print(f'Error occurred during transaction:\n{e}')
            raise

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()

    try:
        # Update date from the last_update table based on max date on the file
        old_last_update = conn.execute(text(f'SELECT date FROM {SCHEMA}.last_update')).fetchall()[0][0]
        new_last_update = last_update
        if new_last_update > old_last_update:
            conn.execute(text(f"UPDATE {SCHEMA}.last_update SET date = '{new_last_update}';"))

        # Commit the transaction
        trans.commit()

    except Exception as e:
        # Rollback the transaction on exception
        print('!!! [ERROR IN DATABASE QUERIES] !!!')
        trans.rollback()
        print('Transaction has been rolled back')
        print(f'Error occurred during transaction:\n{e}')
        raise